In [1]:
import sys 
sys.path.append('../..')

In [2]:
from main_utils import LLM_EmbedData

/opt/conda/envs/py38/lib/python3.8/site-packages/thefuzz/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
data = LLM_EmbedData(
    data_source_yaml_path='../../datasets/taxonomy_embedding/wish-newtax-v1.2.1.yaml', 
    model_name='sentence-transformers/sentence-t5-large'
)

In [4]:
data.hparams.raw_cache_dir_folder

'/data/junwang/.cache/general/0c97db8b8ffd84b0e0bf0ca49140395f'

In [5]:
!ls /data/junwang/.cache/general/0c97db8b8ffd84b0e0bf0ca49140395f

test.json.gz  test.json.gz.lock


In [6]:
import pandas as pd
import os

In [7]:
df = pd.read_json(
    os.path.join(data.hparams.raw_cache_dir_folder, 'test.json.gz'), 
    lines=True)

In [8]:
df

,text_input,text_output
0,Embed taxonomy: security & protection > buildi...,Embed taxonomy: security & protection > buildi...
1,Embed taxonomy: security & protection > door i...,Embed taxonomy: security & protection > door i...
2,Embed taxonomy: security & protection > emerge...,Embed taxonomy: security & protection > emerge...
3,Embed taxonomy: security & protection > fire p...,Embed taxonomy: security & protection > fire p...
4,Embed taxonomy: security & protection > iot de...,Embed taxonomy: security & protection > iot de...
...,...,...
6032,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6033,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6034,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...
6035,Embed taxonomy: women's clothing > underwear &...,Embed taxonomy: women's clothing > underwear &...


In [9]:
df_infer = pd.read_json('../../models/product_title_embedding/version_4/epoch=4-step=398293--wish-newtax-v1.2.1.json', lines=True)

In [10]:
import numpy as np

In [11]:
embs = np.array(df_infer.sort_values('batch_indices').embedding.tolist())

In [39]:
import umap
mapper = umap.UMAP().fit(embs / (embs**2).sum(1, keepdims=True)**.5)

In [40]:
import umap.plot
umap.plot.output_notebook()

Loading BokehJS ...

In [41]:
hover_data = pd.DataFrame({'index': np.arange(len(df)),
                           'text': df['text_input'].apply(lambda x: x.split(': ')[1])
                           })

In [42]:
fig = umap.plot.interactive(
    mapper, 
    labels=df['text_output'].apply(lambda x: x.split(': ')[1].split(' > ')[0]),
    point_size=5,
    hover_data=hover_data
)

In [43]:
umap.plot.show(fig)

In [12]:
import torch
from torch import nn

In [13]:
t = nn.functional.normalize(torch.tensor(embs))

In [14]:
from tqdm import tqdm

In [18]:
t = t.cuda()
sims = []
for chunk in tqdm(torch.chunk(t, 100)):
    sims_chunk = -torch.pow(
        torch.clamp(
            # m x 1 x d - n x d -> m x n x d https://pytorch.org/docs/stable/notes/broadcasting.html
            # normalize is important for optimization
            chunk.cuda().unsqueeze(1) - \
                t, min=0
        ), 2
    ).sum(2).detach().cpu().numpy()
    sims.append(sims_chunk)
sims = np.vstack(sims)

100%|██████████| 99/99 [00:03<00:00, 28.95it/s]


In [17]:
sims_euclid = []
for chunk in tqdm(torch.chunk(t, 100)):
    sims_chunk = -torch.pow(
        chunk.cuda().unsqueeze(1) - t, 2
    ).sum(2).detach().cpu().numpy()
    sims_euclid.append(sims_chunk)
sims_euclid = np.vstack(sims_euclid)

100%|██████████| 99/99 [00:02<00:00, 41.55it/s]


In [19]:
import matplotlib.pyplot as plt

In [19]:
sims.shape

(6037, 6037)

In [20]:
sims_euclid.shape

(6037, 6037)

In [21]:
top_sims, top_inds = torch.tensor(sims).topk(len(sims))
top_sims_euclid, top_inds_euclid = torch.tensor(sims_euclid).topk(len(sims_euclid))

In [22]:
df['category'] = df.text_output.apply(lambda x: x.split(": ")[1].split(" > ")).tolist()

In [23]:
top_inds_l1 = top_inds[df[df['category'].apply(len) == 1].index.to_numpy()]
top_sims_l1 = top_sims[df[df['category'].apply(len) == 1].index.to_numpy()]

In [27]:
top_inds_l1_euclid = top_inds_euclid[df[df['category'].apply(len) == 1].index.to_numpy()]
top_sims_l1_euclid = top_sims_euclid[df[df['category'].apply(len) == 1].index.to_numpy()]

In [24]:
pd.set_option('display.max_colwidth', None)

In [25]:
from pprint import pprint

In [26]:
for ind in range(len(top_inds_l1)):
    tmp = df.loc[top_inds_l1[ind]].copy()
    tmp['rank'] = list(range(len(tmp)))
    tmp['sims'] = top_sims_l1[ind]
    l1_cat = df.loc[top_inds_l1[ind][0].item()].category[0]
    max_rank = tmp[tmp.category.apply(lambda x: x[0] == l1_cat)]['rank'].max()
    tmp = tmp[tmp['rank'] <= max_rank]
    tmp_noise = tmp[tmp.category.apply(lambda x: x[0] != l1_cat)]
    print(f'> {l1_cat}')
    pprint(tmp_noise.head(5)['category'].tolist())

> apparel accessories
[['home improvement'],
 ['home & garden',
  'arts, crafts & sewing',
  'apparel sewing & fabric',
  'buckles & hooks'],
 ['luggage & bags', 'bag parts & accessories'],
 ['novelty & special use'],
 ['jewelry & accessories', 'customized jewelry', 'customized hair jewelry']]
> automobiles & motorcycles
[['sports',
  'roller skates, skateboards & scooters',
  'scooters',
  'gas scooters'],
 ['sports',
  'roller skates, skateboards & scooters',
  'scooters',
  'electric scooters'],
 ['sports', 'cycling', 'electric bicycle'],
 ['sports',
  'roller skates, skateboards & scooters',
  'scooters',
  'handicapped scooters'],
 ['sports', 'cycling', 'electric bicycle part', 'conversion kit']]
> beauty & health
[['mother & kids', 'feeding', 'breast shells & creams'],
 ['home & garden', 'arts, crafts & sewing', 'soap making', 'soap scents'],
 ['home improvement',
  'home appliances',
  'kitchen appliances',
  'reusable coffee capsules & pods'],
 ['home & garden',
  'home decor',

In [71]:
df[df.category.apply(len) == 2].sample(5)[['text_output']].to_dict('records')

[{'text_output': 'Embed taxonomy: toys & hobbies > diecast & toy vehicles'},
 {'text_output': 'Embed taxonomy: consumer electronics > earphones & headphones'},
 {'text_output': "Embed taxonomy: luggage & bags > men's bags"},
 {'text_output': 'Embed taxonomy: computer & office > servers'},
 {'text_output': 'Embed taxonomy: security & protection > security inspection device'}]

In [104]:
embs_n = embs / (embs**2).sum(1, keepdims=True)**.5

In [107]:
cat_a = 'sports'
cat_b = "shoes"
emb_a = embs[df[df.category.apply(lambda x: ' > '.join(x) == cat_a)].index.to_numpy()]
emb_b = embs[df[df.category.apply(lambda x: ' > '.join(x) == cat_b)].index.to_numpy()]
emb_a_intersect_b = np.maximum(emb_a, emb_b)
emb_a_union_b = np.minimum(emb_a, emb_b)
emb_a_intersect_b = emb_a_intersect_b / (emb_a_intersect_b**2).sum(1, keepdims=True)**.5
emb_a_union_b = np.minimum(emb_a, emb_b) / (emb_a_union_b**2).sum(1, keepdims=True)**.5

In [108]:
df.loc[((emb_a_intersect_b - embs_n)**2).sum(1).argsort()].head(5)[['category']].to_dict('records')

[{'category': ['sports']},
 {'category': ['sports', 'sneakers']},
 {'category': ['sports', 'sneakers', 'fencing shoes']},
 {'category': ['sports', 'sneakers', 'toning shoes']},
 {'category': ['sports', 'sneakers', 'baseball shoes']}]

In [111]:
cat_a = 'novelty & special use > costumes & accessories > shoes'
cat_b = 'sports > sneakers > walking shoes'
emb_a = embs[df[df.category.apply(lambda x: ' > '.join(x) == cat_a)].index.to_numpy()]
emb_b = embs[df[df.category.apply(lambda x: ' > '.join(x) == cat_b)].index.to_numpy()]
emb_a_intersect_b = np.maximum(emb_a, emb_b)
emb_a_union_b = np.minimum(emb_a, emb_b)
emb_a_intersect_b = emb_a_intersect_b / (emb_a_intersect_b**2).sum(1, keepdims=True)**.5
emb_a_union_b = np.minimum(emb_a, emb_b) / (emb_a_union_b**2).sum(1, keepdims=True)**.5

In [112]:
df.loc[((emb_a_union_b - embs_n)**2).sum(1).argsort()].head(5)[['category']].to_dict('records')

[{'category': ['shoes']},
 {'category': ['novelty & special use', 'costumes & accessories', 'shoes']},
 {'category': ['sports', 'sneakers', 'walking shoes']},
 {'category': ['sports',
   'roller skates, skateboards & scooters',
   'skateboarding']},
 {'category': ['sports', 'sneakers', 'skateboarding shoes']}]